# LOS Tabular

## Setup

In [3]:
# First install package from terminal:
!pip install -U pip
!pip install -U setuptools wheel
!pip install autogluon  # autogluon==0.4.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached autogluon-0.6.0-py3-none-any.whl (9.8 kB)
  Using cached autogluon.timeseries-0.6.0-py3-none-any.whl (101 kB)
  Using cached autogluon.tabular-0.6.0-py3-none-any.whl (285 kB)
  Using cached autogluon.core-0.6.0-py3-none-any.whl (224 kB)
  Using cached autogluon.features-0.6.0-py3-none-any.whl (59 kB)
  Using cached autogluon.vision-0.6.0-py3-none-any.whl (49 kB)
  Using cached autogluon.text-0.6.0-py3-none-any.whl (62 kB)
  Using cached autogluon.multimodal-0.6.0-py3-none-any.whl (279 kB)
  Using cached dask-2021.11.2-py3-none-any.whl (1.0 MB)
  Using cached distributed-2021.11.2-py3-none-any.whl (802 kB)
  Using cached boto3-1.26.16-py3-none-any.whl (132 kB)
  Using cached autog

In [4]:
# import all required modules
import pandas as pd
import numpy as np
import os
import re
import random
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_rows', None)  ###
pd.set_option('display.max_columns', None)  ###
pd.set_option('display.width', None)  ###
pd.set_option('display.max_colwidth', None)  ###

import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve
from sklearn.metrics import classification_report
from sklearn.metrics import auc
from sklearn.metrics import cohen_kappa_score
from sklearn.inspection import permutation_importance
from autogluon.tabular import TabularPredictor

In [5]:
try:
  from google.colab import drive
  IN_COLAB=True
except:
  IN_COLAB=False

if IN_COLAB:
  print("We're running Colab")

if IN_COLAB:  
  # Mount the Google Drive at mount
  mount='/content/gdrive'
  print("Colab: mounting Google drive on ", mount)
  # connect your colab with the drive
  drive.mount(mount)

 # Switch to the directory on the Google Drive that you want to use
  import os
  path_to_repo = mount + "/MyDrive/MIMIC-III Text Mining/LOS_FINAL/"

else:
  # Setup Repository
  with open("repo_info.txt", "r") as repo_info:
      path_to_repo = repo_info.readline()

  
print(path_to_repo)

path_to_data = f"{path_to_repo}data/"
path_to_raw = f"{path_to_data}raw/"
path_to_processed = f"{path_to_data}processed/"
path_to_lda = f"{path_to_data}lda/"
path_to_icd = f"{path_to_data}icd_codes/"
path_to_models = f"{path_to_repo}models/"
path_to_results = f"{path_to_repo}results/"

We're running Colab
Colab: mounting Google drive on  /content/gdrive
Mounted at /content/gdrive
/content/gdrive/MyDrive/MIMIC-III Text Mining/LOS_FINAL/


## Import the dataset

In [6]:
def load_datasets(method):
    """
    Function to load train, test and validation set based on the chosen method
    method: string for the processing method we want to load
    """
    global path_to_processed
    # load it back
    train = pd.read_feather('{}{}train_{}{}{}{}'.format(path_to_processed, method, seed_tag, preproc_tag, lemma_tag, death_tag))
    test = pd.read_feather('{}{}test_{}{}{}{}'.format(path_to_processed, method, seed_tag, preproc_tag, lemma_tag, death_tag))
    return train, test

In [7]:
# PARAMETERS

session_seed = 42 # set seed for our session
seed_tag = f'_{session_seed}'

random.seed(session_seed)


# Set to True if we want to include deaths
death_incl = False
death_tag = np.where(death_incl,"_death", "")

## Train the Tabular Model (Structured DataSet)

In [8]:
# Model Parameters
label = 'los_cat'
metric = 'roc_auc'

In [9]:
# compute other metrics
def perf_evaluator(y_test, y_pred, y_pred_proba):
    """ Function to display the main classification performance metrics """
    kappa = cohen_kappa_score(y_test, y_pred)
    precision, recall, prc_th = precision_recall_curve(y_test, y_pred_proba)
    prc_auc = auc(recall, precision)
    return kappa, prc_auc

In [10]:
# Iterate over our main methods of vectorization

vect_dict = {'tabular': (False, False),
             'stemming': (False, False),
             'spacy': (True, True)}

for key, value in vect_dict.items():
    print(key)
    # PARAMETERS

    lemmatize = value[0] # set to false if we want to do stemming
    lemma_tag = str(np.where(lemmatize, "_lemma",""))
    spacy = value[1]
    if spacy: lemma_tag = str(np.where(lemmatize, "_lemma_spacy",""))

    preprocessing = True # set to true if we want to clean and perform some preprocessing
    preproc_heavier = True # set to True if we want a heavier preprocessing
    preproc_tag_2 = np.where(preproc_heavier, '_heavier', '')
    preproc_tag = np.where(preprocessing, f'_preproc{preproc_tag_2}', f'{preproc_tag_2}')

    # Iterate over all the methods
    if key != 'tabular':
        method_list = ['frequency', 'onehot','tf_idf']
    else:
        method_list = ['frequency']

    for method in method_list:
        print(method)
        # Load the LDA datasets
        train, test = load_datasets(f'lda_{method}{death_tag}')
        if key == 'tabular':
            # If we are just running the tabular dataset, drop the LDA columns
            lda_topics = [col for col in train.columns if re.match(r"F[0-9]*", col)]
            train.drop(columns = lda_topics, inplace = True)
            test.drop(columns = lda_topics, inplace = True)
            assert len(lda_topics) == 300

        save_path = f'{path_to_models}text{preproc_tag}{lemma_tag}{method}{death_tag}'
        os.makedirs(save_path, exist_ok = True)

        # run the tabular predictor ensemble of models
        predictor = TabularPredictor(label=label, eval_metric=metric,path=save_path)
        predictor.fit(train)

        # evaluate performance on the test set
        per_tab = predictor.evaluate(test)
        print(f"\nTest set performance:\n{per_tab}")
        # save the class and probability predictions
        y_pred = predictor.predict(test)
        y_pred_proba = predictor.predict_proba(test).iloc[:,1]
        perf = perf_evaluator(test['los_cat'], y_pred, y_pred_proba)
        perf_dict = {"Cohen's Kappa": perf[0], "PRC AUC": perf[1]}
        print(f"\nPerformance metrics:\n{perf_dict}")

        # compare the different models
        leaderboard = predictor.leaderboard(test)
        print(f"\nModel Leaderboard:\n{leaderboard}")

        # compute feature importance
        importance = predictor.feature_importance(test)
        # feature importance in percentage
        importance['percent'] = 100*importance['importance']/importance['importance'].max()
        print(f"\nFeature Importance:\n{importance}")
        perf_dict.update(per_tab)
        # save performances
        df_perf = pd.DataFrame.from_dict(perf_dict, orient='index', columns=['performances'])
        df_perf.to_excel(path_to_results+f'{key}/df_perf_{method}{death_tag}.xlsx')
        # save leaderboard
        leaderboard.to_excel(path_to_results+f'{key}/leaderboard_{method}{death_tag}.xlsx')
        # save importance
        importance.to_excel(path_to_results+f'{key}/importance_{method}{death_tag}.xlsx')

tabular
frequency


Beginning AutoGluon training ...
AutoGluon will save models to "/content/gdrive/MyDrive/MIMIC-III Text Mining/LOS_FINAL/models/text_preproc_heavierfrequency/"
AutoGluon Version:  0.6.0
Python Version:     3.7.15
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Fri Aug 26 08:44:51 UTC 2022
Train Data Rows:    24611
Train Data Columns: 46
Label Column: los_cat
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [False, True]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = True, class 0 = False
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    53204.8 MB
	Train Data (Original)  Memory U


Test set performance:
{'roc_auc': 0.9441635626643294, 'accuracy': 0.9471802372826263, 'balanced_accuracy': 0.7051531003505697, 'mcc': 0.5348751697099995, 'f1': 0.5376955903271693, 'precision': 0.7411764705882353, 'recall': 0.421875}

Performance metrics:
{"Cohen's Kappa": 0.5119144639388208, 'PRC AUC': 0.6552421524932525}


Computing feature importance via permutation shuffling for 46 features using 5000 rows with 5 shuffle sets...


                  model  score_test  score_val  pred_time_test  pred_time_val   fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   WeightedEnsemble_L2    0.944164   0.947699        0.985626       0.613746  72.914350                 0.005695                0.000650           0.712254            2       True         14
1            LightGBMXT    0.942363   0.941051        0.060758       0.056628   2.971784                 0.060758                0.056628           2.971784            1       True          3
2              CatBoost    0.942151   0.942311        0.026469       0.015857   9.102162                 0.026469                0.015857           9.102162            1       True          7
3              LightGBM    0.939862   0.943836        0.025571       0.030734   1.252226                 0.025571                0.030734           1.252226            1       True          4
4         LightGBMLarge    0.939767   0.

...


Test set performance:
{'roc_auc': 0.9540104231876799, 'accuracy': 0.9501056395254347, 'balanced_accuracy': 0.713929666958808, 'mcc': 0.5619936115772544, 'f1': 0.5608011444921316, 'precision': 0.7808764940239044, 'recall': 0.4375}

Performance metrics:
{"Cohen's Kappa": 0.5365683599150255, 'PRC AUC': 0.6896537155769118}


Computing feature importance via permutation shuffling for 346 features using 5000 rows with 5 shuffle sets...


                  model  score_test  score_val  pred_time_test  pred_time_val    fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   WeightedEnsemble_L2    0.954010   0.958579        2.152935       1.184751  148.810848                 0.008920                0.000807           0.813180            2       True         14
1            LightGBMXT    0.951707   0.952669        0.098294       0.025141    5.956255                 0.098294                0.025141           5.956255            1       True          3
2              CatBoost    0.950442   0.954894        0.049232       0.032575   51.169433                 0.049232                0.032575          51.169433            1       True          7
3       NeuralNetFastAI    0.949247   0.945986        0.289735       0.178510   31.906057                 0.289735                0.178510          31.906057            1       True         10
4        NeuralNetTorch    0.948618

	3676.79s	= Expected runtime (735.36s per shuffle set)
	1876.69s	= Actual runtime (Completed 5 of 5 shuffle sets)



Feature Importance:
                      importance    stddev   p_value  n      p99_high  \
urea_n_max          2.676254e-02  0.002712  0.000012  5  3.234729e-02   
platelets_min       2.317750e-02  0.002924  0.000030  5  2.919710e-02   
platelets_max       1.785079e-02  0.003304  0.000135  5  2.465285e-02   
urea_n_min          1.260551e-02  0.000781  0.000002  5  1.421360e-02   
F59                 6.488700e-03  0.000708  0.000017  5  7.946028e-03   
F205                4.661517e-03  0.001152  0.000413  5  7.033221e-03   
platelets_mean      2.857339e-03  0.001086  0.002084  5  5.092804e-03   
temp_max            2.626842e-03  0.001033  0.002359  5  4.753057e-03   
type_stay           2.342231e-03  0.001291  0.007704  5  5.001403e-03   
F198                1.901800e-03  0.000243  0.000031  5  2.401390e-03   
dest_discharge      1.658785e-03  0.000670  0.002597  5  3.037543e-03   
F122                1.552509e-03  0.000424  0.000604  5  2.424806e-03   
sofa                1.386918e-